In [3]:
from sqlalchemy import create_engine
from sqlalchemy import text
from db_auth import getConnectionInfo

login = getConnectionInfo()

username=login['username']
password=login['password']
host=login['host']
port=login['port']
db_name=login['db_name']

engine = create_engine('postgresql://'+username+':'+password+'@'+host+':'+port+'/'+db_name, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2021-11-17 11:14:48,484 INFO sqlalchemy.engine.Engine select version()
2021-11-17 11:14:48,485 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine [generated in 0.00077s] {}
[('hello world',)]
2021-11-17 11:14:48,486 INFO sqlalchemy.engine.Engine ROLLBACK
